# Tutorial: Converting Medical Images (Voxels) to Solid Polygons for Finite Element Analysis
### Overview
This tutorial guides you through the process of converting medical imaging data (CT, MRI) from voxel format into solid polygon meshes suitable for finite element analysis (FEA). This workflow is essential for biomechanical simulations of bones, organs, and other anatomical structures.

### Table of Contents

1. Introduction
2. Prerequisites
3. Workflow Overview
4. Step 1: Loading Medical Images
5. Step 2: Image Segmentation
6. Step 3: Surface Extraction
7. Step 4: Mesh Smoothing
8. Step 5: Mesh Decimation
9. Step 6: Creating Solid Volume Mesh
10. Step 7: Exporting for FEA
11. Complete Pipeline Example
12. Best Practices

## Introduction
Medical imaging modalities like CT and MRI produce 3D volumetric data represented as voxels (3D pixels). To perform finite element analysis, we need to convert this voxel data into:

1. **Surface mesh**: A polygon representation of the object's boundary
2. **Volume mesh**: A solid mesh with tetrahedral or hexahedral elements

## Prerequisites
Required Python Libraries

In [ ]:
pip install numpy scipy scikit-image trimesh pymeshlab pydicom nibabel
# For tetrahedral meshing
pip install pygalmesh
# Optional: for visualization
pip install pyvista matplotlib

Software Tools (Alternative/Complementary)

- 3D Slicer: Free, open-source for segmentation
- ITK-SNAP: Segmentation and visualization
- Meshmixer: Mesh editing and repair
- GMSH: Advanced mesh generation
- TetGen: Tetrahedral mesh generation

Medical Image (DICOM/NIfTI)

         ↓
         
    Segmentation (Binary mask)
         ↓
         
    Surface Extraction (Marching Cubes)
         ↓
         
    Mesh Smoothing
         ↓
         
    Mesh Decimation (Optional)
         ↓
         
    Volume Mesh Generation
         ↓
         
    FEA Software (ANSYS, Abaqus, FEBio, etc.)

## Step 1: Loading Medical Images
Loading DICOM Files

In [1]:
import pydicom
import numpy as np
from pathlib import Path

def load_dicom_series(dicom_folder):
    """
    Load a series of DICOM files and create a 3D volume.
    
    Parameters:
    -----------
    dicom_folder : str
        Path to folder containing DICOM files
    
    Returns:
    --------
    volume : ndarray
        3D numpy array of image data
    spacing : tuple
        Voxel spacing (x, y, z) in mm
    """

    # Get all DICOM files
    dicom_files = sorted(Path(dicom_folder).glob('*.dcm'))
    
    # Read first file to get metadata
    first_slice = pydicom.dcmread(dicom_files[0])
    
    # Get image dimensions
    img_shape = (len(dicom_files), 
                 first_slice.Rows, 
                 first_slice.Columns)
    
    # Get voxel spacing
    pixel_spacing = first_slice.PixelSpacing
    slice_thickness = first_slice.SliceThickness
    spacing = (float(pixel_spacing[0]), 
               float(pixel_spacing[1]), 
               float(slice_thickness))
    
    # Initialize volume array
    volume = np.zeros(img_shape, dtype=np.int16)
    
    # Load all slices
    for i, filepath in enumerate(dicom_files):
        ds = pydicom.dcmread(filepath)
        volume[i, :, :] = ds.pixel_array
    
    return volume, spacing


dcm_path = '/Users/pasit/Documents/dataset/DIRLAB/dcm/Case4/DirLab4D_4_10'
volume, spacing = load_dicom_series(dcm_path)
print(f"Volume shape: {volume.shape}")
print(f"Voxel spacing (mm): {spacing}")


Volume shape: (99, 256, 256)
Voxel spacing (mm): (1.13, 1.13, 2.5)


## Step 2: Image Segmentation
Segmentation identifies the region of interest (bone, organ, etc.) in the image.

### Threshold-Based Segmentation

In [2]:
def threshold_segmentation(volume, lower_threshold, upper_threshold):
    """
    Simple threshold-based segmentation.
    
    Parameters:
    -----------
    volume : ndarray
        3D image volume
    lower_threshold : float
        Lower intensity threshold
    upper_threshold : float
        Upper intensity threshold
    
    Returns:
    --------
    mask : ndarray (bool)
        Binary segmentation mask
    """
    mask = (volume >= lower_threshold) & (volume <= upper_threshold)
    return mask

# Example: Segment bone from CT (Hounsfield units)
bone_mask = threshold_segmentation(volume, lower_threshold=200, upper_threshold=3000)

### Advanced Segmentation with Morphological Operations

In [3]:
from scipy import ndimage
from skimage import morphology

def clean_segmentation(mask, min_size=1000):
    """
    Clean segmentation mask using morphological operations.
    
    Parameters:
    -----------
    mask : ndarray (bool)
        Binary segmentation mask
    min_size : int
        Minimum component size to keep
    
    Returns:
    --------
    cleaned_mask : ndarray (bool)
        Cleaned binary mask
    """
    # Remove small objects
    cleaned = morphology.remove_small_objects(mask, min_size=min_size)
    
    # Fill holes
    cleaned = ndimage.binary_fill_holes(cleaned)
    
    # Morphological closing to smooth boundaries
    cleaned = ndimage.binary_closing(cleaned, structure=np.ones((3, 3, 3)))
    
    return cleaned

# Apply cleaning
bone_mask_cleaned = clean_segmentation(bone_mask, min_size=5000)

/var/folders/94/sw8gj1ps0nl62j4mdtv5wtnc0000gn/T/ipykernel_19291/1984659732.py:21: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later). To avoid this warning, please use the parameter `max_size` instead. For more details, see the documentation of `remove_small_objects`. Note that the new threshold removes objects smaller than **or equal to** its value, while the previous parameter only removed smaller ones.
  cleaned = morphology.remove_small_objects(mask, min_size=min_size)


## Step 3: Surface Extraction
Use the Marching Cubes algorithm to extract a triangular surface mesh from the voxel data.

### Using scikit-image

In [4]:
from skimage import measure

def extract_surface(mask, spacing=(1.0, 1.0, 1.0), level=0.5):
    """
    Extract surface mesh using Marching Cubes.
    
    Parameters:
    -----------
    mask : ndarray (bool)
        Binary segmentation mask
    spacing : tuple
        Voxel spacing (x, y, z) in mm
    level : float
        Iso-surface level
    
    Returns:
    --------
    vertices : ndarray
        Mesh vertices (N, 3)
    faces : ndarray
        Mesh faces (M, 3)
    normals : ndarray
        Vertex normals (N, 3)
    """
    # Run Marching Cubes
    vertices, faces, normals, values = measure.marching_cubes(
        mask.astype(float), 
        level=level,
        spacing=spacing
    )
    
    return vertices, faces, normals

# Extract surface
vertices, faces, normals = extract_surface(bone_mask_cleaned, spacing=spacing)
print(f"Vertices: {len(vertices)}, Faces: {len(faces)}")

Vertices: 231724, Faces: 464820


### Save as STL

In [5]:
import trimesh

def save_stl(vertices, faces, filename):
    """
    Save mesh as STL file.
    
    Parameters:
    -----------
    vertices : ndarray
        Mesh vertices
    faces : ndarray
        Mesh faces
    filename : str
        Output filename
    """
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    mesh.export(filename)

save_stl(vertices, faces, 'bone_surface.stl')

To view STL file
https://trellis3d.co/online-viewer/stl

## Step 4: Mesh Smoothing
Smooth the mesh to remove staircase artifacts from voxel data.

### Laplacian Smoothing

In [6]:
import pymeshlab

def smooth_mesh(input_file, output_file, iterations=10):
    """
    Smooth mesh using Laplacian smoothing.
    
    Parameters:
    -----------
    input_file : str
        Input mesh file
    output_file : str
        Output mesh file
    iterations : int
        Number of smoothing iterations
    """
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(input_file)
    
    # Apply Laplacian smoothing
    ms.apply_filter('apply_coord_laplacian_smoothing', 
                     stepsmoothnum=iterations,
                     cotangentweight=False)
    
    ms.save_current_mesh(output_file)

# Smooth the mesh
smooth_mesh('bone_surface.stl', 'bone_smoothed.stl', iterations=5)

### Taubin Smoothing (Better preserves features)

In [7]:
def taubin_smooth(input_file, output_file, iterations=10, lambda_val=0.5, mu_val=-0.53):
    """
    Apply Taubin smoothing (better feature preservation).
    
    Parameters:
    -----------
    input_file : str
        Input mesh file
    output_file : str
        Output mesh file
    iterations : int
        Number of smoothing iterations
    lambda_val : float
        Lambda parameter
    mu_val : float
        Mu parameter (should be negative)
    """
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(input_file)
    
    ms.apply_filter('apply_coord_taubin_smoothing',
                     stepsmoothnum=iterations,
                     lambda_=lambda_val,
                     mu=mu_val)
    
    ms.save_current_mesh(output_file)

taubin_smooth('bone_surface.stl', 'bone_taubin.stl', iterations=10)

## Step 5: Mesh Decimation
Reduce mesh complexity while preserving geometry.

In [8]:
def decimate_mesh(input_file, output_file, target_faces=10000):
    """
    Reduce mesh complexity using quadric edge collapse decimation.
    
    Parameters:
    -----------
    input_file : str
        Input mesh file
    output_file : str
        Output mesh file
    target_faces : int
        Target number of faces
    """
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(input_file)
    
    # Get current face count
    current_faces = ms.current_mesh().face_number()
    target_percentage = target_faces / current_faces
    
    # Apply decimation
    ms.apply_filter('meshing_decimation_quadric_edge_collapse',
                     targetfacenum=target_faces,
                     preserveboundary=True,
                     preservenormal=True,
                     preservetopology=True)
    
    ms.save_current_mesh(output_file)
    
    print(f"Reduced from {current_faces} to {ms.current_mesh().face_number()} faces")

decimate_mesh('bone_taubin.stl', 'bone_decimated.stl', target_faces=20000)

Reduced from 464820 to 20000 faces


## Step 6: Creating Solid Volume Mesh
Generate tetrahedral volume mesh for FEA.

### Using pygalmesh

In [9]:
import pygalmesh

def create_volume_mesh(input_stl, output_mesh, max_cell_size=2.0, max_edge_size=1.0):
    """
    Create tetrahedral volume mesh from surface mesh.
    
    Parameters:
    -----------
    input_stl : str
        Input surface mesh file (.stl, .obj)
    output_mesh : str
        Output volume mesh file (.mesh, .vtk, .vtu)
    max_cell_size : float
        Maximum cell circumradius
    max_edge_size : float
        Maximum edge length
    """
    mesh = pygalmesh.generate_from_surface_mesh(
        input_stl,
        max_cell_circumradius=max_cell_size,
        max_edge_size_at_feature_edges=max_edge_size,
        verbose=True
    )
    
    mesh.write(output_mesh)

# Generate volume mesh
create_volume_mesh('bone_decimated.stl', 'bone_volume.vtk', 
                   max_cell_size=3.0, max_edge_size=2.0)

ModuleNotFoundError: No module named 'pygalmesh'